**Importing the imporant libraries that will be used thoughout the notebook**

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt
import re
import string

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
nltk.download('stopwords')
nltk.download('vader_lexicon')


from collections import Counter

from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
import plotly.express as px

sns.set(style="darkgrid")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


**Data Collection**

In [45]:
df = pd.read_csv("/content/Copy of NigeriaEndSars data - NigeriaEndSars data.csv (1).csv")
needed_columns=['username','date','content','Labels']
manual_df = df[:702].copy()
manual_df= manual_df[needed_columns]
manual_df.head()

,username,date,content,Labels
0,hurbhiaghelee,2021-04-22 18:28:26+00:00,This is the highest form of intimidation\nWhat...,Negative
1,AlukoJoseph5,2021-04-22 15:07:05+00:00,Why is it taking a life long time for @MBuhari...,Positive
2,omofuma_Elijah,2021-04-22 10:13:20+00:00,"Even Desmond Elliot is quite , ,,, what is tha...",Negative
3,FelixOdohoedi,2021-04-22 09:48:04+00:00,When will CBN freeze the account of @DrIsaPant...,Positive
4,adekzee1,2021-04-22 09:31:40+00:00,@Obiejazzyup @YeleSowore @MBuhari @EndSars @mb...,Positive


In [46]:
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}

manual_df["Labels"] = manual_df["Labels"].map(label_map)


In [47]:
manual_df.isnull().sum()

username    0
date        0
content     0
Labels      0
dtype: int64

**convert username type from object to category for assigning the numbers after it**

In [48]:
df.username=df.username.astype('category')

**Assign a unique numerical code to each category**

In [49]:
df.username=df.username.cat.codes
df.date=pd.to_datetime(df.date).dt.date # it will give only the date

# assigning content column to 'texts' variable
texts=df.content

**Data Preprocessing**

In [50]:
#Removing URLS from tweets
remove_url=lambda x:re.sub(r'http\S+','',str(x))

#converting to all tweets to lowercase
to_lower=lambda x: x.lower()

#Removing punctuations
remove_puncs= lambda x:x.translate(str.maketrans('','',string.punctuation))

#Removing stopwords
more_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
stop_words=set(stopwords.words('english')) #nltk package
stop_words.update(more_words)

remove_words=lambda x: ' '.join([word for word in x.split() if word not in stop_words]) #.join is from package string


texts_lr=texts.apply(remove_url)
texts_lr_lc=texts_lr.apply(to_lower)
texts_lr_lc_np=texts_lr_lc.apply(remove_puncs)
texts_lr_lc_np_ns=r=texts_lr_lc_np.apply(remove_words)
df['content'] = texts_lr_lc_np_ns

**Putting Cleaned Text in the main DataFrame**

In [51]:
df.text=texts_lr_lc_np_ns
display(df.head(5))

<ipython-input-51-f517616bf0d2>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.text=texts_lr_lc_np_ns


,Unnamed: 0,url,date,content,id,username,outlinks,outlinksss,tcooutlinks,tcooutlinksss,Labels
0,0,https://twitter.com/hurbhiaghelee/status/13852...,2021-04-22,highest form intimidation offence commit give ...,1.385299e+18,4928,['https://twitter.com/AmaiweLove/status/138519...,https://twitter.com/AmaiweLove/status/13851900...,['https://t.co/xxdTK0BmQB'],https://t.co/xxdTK0BmQB,Negative
1,1,https://twitter.com/AlukoJoseph5/status/138524...,2021-04-22,taking life long time mbuhari sack drisapantam...,1.385249e+18,382,[],NaN,[],NaN,Positive
2,2,https://twitter.com/omofuma_Elijah/status/1385...,2021-04-22,even desmond elliot quite womans name look lik...,1.385175e+18,5768,['https://twitter.com/vhic_tore/status/1385141...,https://twitter.com/vhic_tore/status/138514152...,['https://t.co/RsVQ4N8LwM'],https://t.co/RsVQ4N8LwM,Negative
3,3,https://twitter.com/FelixOdohoedi/status/13851...,2021-04-22,cbn freeze account drisapantami cbn quick free...,1.385169e+18,1441,['https://twitter.com/Imudia_se2/status/138495...,https://twitter.com/Imudia_se2/status/13849543...,['https://t.co/r3vxrbkLdn'],https://t.co/r3vxrbkLdn,Positive
4,4,https://twitter.com/adekzee1/status/1385164392...,2021-04-22,obiejazzyup yelesowore mbuhari endsars mbujari...,1.385164e+18,4045,[],NaN,[],NaN,Positive


**Additional Data Cleaning**

In [52]:
def clean_text(text):
    '''remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
df['content'] = df['content'].apply(lambda x: clean_text(x))



# function to remove emoticons, symbols or flags by their codes
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['content']=df['content'].apply(lambda x: remove_emoji(x))


df

,Unnamed: 0,url,date,content,id,username,outlinks,outlinksss,tcooutlinks,tcooutlinksss,Labels
0,0,https://twitter.com/hurbhiaghelee/status/13852...,2021-04-22,highest form intimidation offence commit give ...,1.385299e+18,4928,['https://twitter.com/AmaiweLove/status/138519...,https://twitter.com/AmaiweLove/status/13851900...,['https://t.co/xxdTK0BmQB'],https://t.co/xxdTK0BmQB,Negative
1,1,https://twitter.com/AlukoJoseph5/status/138524...,2021-04-22,taking life long time mbuhari sack drisapantam...,1.385249e+18,382,[],NaN,[],NaN,Positive
2,2,https://twitter.com/omofuma_Elijah/status/1385...,2021-04-22,even desmond elliot quite womans name look lik...,1.385175e+18,5768,['https://twitter.com/vhic_tore/status/1385141...,https://twitter.com/vhic_tore/status/138514152...,['https://t.co/RsVQ4N8LwM'],https://t.co/RsVQ4N8LwM,Negative
3,3,https://twitter.com/FelixOdohoedi/status/13851...,2021-04-22,cbn freeze account drisapantami cbn quick free...,1.385169e+18,1441,['https://twitter.com/Imudia_se2/status/138495...,https://twitter.com/Imudia_se2/status/13849543...,['https://t.co/r3vxrbkLdn'],https://t.co/r3vxrbkLdn,Positive
4,4,https://twitter.com/adekzee1/status/1385164392...,2021-04-22,obiejazzyup yelesowore mbuhari endsars mbujari...,1.385164e+18,4045,[],NaN,[],NaN,Positive
...,...,...,...,...,...,...,...,...,...,...,...
9732,9732,https://twitter.com/ReallyPetit/status/9374579...,2017-12-03,yomishogunle policengpcrru policeng much apolo...,9.374579e+17,3220,[],NaN,[],NaN,NaN
9733,9733,https://twitter.com/D_Ogiamien/status/93737939...,2017-12-03,government policeng policengpcrru pretend seei...,9.373794e+17,997,[],NaN,[],NaN,NaN
9734,9734,https://twitter.com/kingsley_obiako/status/937...,2017-12-03,endsars issues ending sars issue reorientation...,9.372872e+17,5258,[],NaN,[],NaN,NaN
9735,9735,https://twitter.com/Eze_Plano_Rich/status/9372...,2017-12-03,u handsome suspect wear new shoe suspect holdi...,9.372639e+17,1392,[],NaN,[],NaN,NaN


**Annotations/Sentiment Analysis**

In [53]:
# getting polarity scores of tweets and storing them in variable 'sentiment_scores'
sid=SentimentIntensityAnalyzer()
ps=lambda x:sid.polarity_scores(x)
sentiment_scores=df.content.apply(ps)
sentiment_scores

0       {'neg': 0.248, 'neu': 0.644, 'pos': 0.109, 'co...
1       {'neg': 0.305, 'neu': 0.495, 'pos': 0.201, 'co...
2       {'neg': 0.152, 'neu': 0.758, 'pos': 0.09, 'com...
3       {'neg': 0.177, 'neu': 0.731, 'pos': 0.092, 'co...
4       {'neg': 0.184, 'neu': 0.559, 'pos': 0.257, 'co...
                              ...                        
9732    {'neg': 0.09, 'neu': 0.667, 'pos': 0.242, 'com...
9733    {'neg': 0.149, 'neu': 0.851, 'pos': 0.0, 'comp...
9734    {'neg': 0.0, 'neu': 0.828, 'pos': 0.172, 'comp...
9735    {'neg': 0.35, 'neu': 0.504, 'pos': 0.146, 'com...
9736    {'neg': 0.17, 'neu': 0.764, 'pos': 0.066, 'com...
Name: content, Length: 9737, dtype: object

In [54]:
# create the data frame of negative, neutral, positive and compound polarity scores
sentiment_df=pd.DataFrame(data=list(sentiment_scores))
labelize=lambda x:'neutral' if x==0 else('positive' if x>0 else 'negative')
sentiment_df['label']=sentiment_df.compound.apply(labelize)
display(sentiment_df.head(10))

,neg,neu,pos,compound,label
0,0.248,0.644,0.109,-0.4215,negative
1,0.305,0.495,0.201,-0.7003,negative
2,0.152,0.758,0.090,-0.1779,negative
3,0.177,0.731,0.092,-0.6369,negative
4,0.184,0.559,0.257,0.3818,positive
5,0.269,0.493,0.238,-0.1779,negative
6,0.000,1.000,0.000,0.0000,neutral
7,0.407,0.543,0.050,-0.7845,negative
8,0.220,0.680,0.100,-0.4588,negative
9,0.000,1.000,0.000,0.0000,neutral


In [55]:
data=df.join(sentiment_df.label)
sentiment_df = df.join(sentiment_df)
display(data.head(5))

,Unnamed: 0,url,date,content,id,username,outlinks,outlinksss,tcooutlinks,tcooutlinksss,Labels,label
0,0,https://twitter.com/hurbhiaghelee/status/13852...,2021-04-22,highest form intimidation offence commit give ...,1.385299e+18,4928,['https://twitter.com/AmaiweLove/status/138519...,https://twitter.com/AmaiweLove/status/13851900...,['https://t.co/xxdTK0BmQB'],https://t.co/xxdTK0BmQB,Negative,negative
1,1,https://twitter.com/AlukoJoseph5/status/138524...,2021-04-22,taking life long time mbuhari sack drisapantam...,1.385249e+18,382,[],NaN,[],NaN,Positive,negative
2,2,https://twitter.com/omofuma_Elijah/status/1385...,2021-04-22,even desmond elliot quite womans name look lik...,1.385175e+18,5768,['https://twitter.com/vhic_tore/status/1385141...,https://twitter.com/vhic_tore/status/138514152...,['https://t.co/RsVQ4N8LwM'],https://t.co/RsVQ4N8LwM,Negative,negative
3,3,https://twitter.com/FelixOdohoedi/status/13851...,2021-04-22,cbn freeze account drisapantami cbn quick free...,1.385169e+18,1441,['https://twitter.com/Imudia_se2/status/138495...,https://twitter.com/Imudia_se2/status/13849543...,['https://t.co/r3vxrbkLdn'],https://t.co/r3vxrbkLdn,Positive,negative
4,4,https://twitter.com/adekzee1/status/1385164392...,2021-04-22,obiejazzyup yelesowore mbuhari endsars mbujari...,1.385164e+18,4045,[],NaN,[],NaN,Positive,positive


In [56]:
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}

sentiment_df["label"] = sentiment_df["label"].map(label_map)


In [57]:
display(sentiment_df["label"])
display(manual_df['Labels'].astype('int'))

0       0
1       0
2       0
3       0
4       2
       ..
9732    2
9733    0
9734    2
9735    0
9736    0
Name: label, Length: 9737, dtype: int64

0      0
1      2
2      0
3      2
4      2
      ..
697    2
698    1
699    1
700    2
701    1
Name: Labels, Length: 702, dtype: int64

In [58]:
from sklearn.metrics import accuracy_score, log_loss, f1_score

In [63]:
model_predictions = sentiment_df['label'][:702]
Ground_truth = manual_df["Labels"].astype('int')
print(accuracy_score(Ground_truth, model_predictions))
#print(log_loss(Ground_truth, model_predictions))
print(f1_score(Ground_truth,model_predictions,average = "macro"))

0.5128205128205128
0.5138101647671781


In [64]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Ground_truth, model_predictions))
print(confusion_matrix(Ground_truth, model_predictions))

              precision    recall  f1-score   support

           0       0.31      0.65      0.42       133
           1       0.62      0.62      0.62       210
           2       0.68      0.40      0.50       359

    accuracy                           0.51       702
   macro avg       0.54      0.56      0.51       702
weighted avg       0.59      0.51      0.52       702

[[ 87  23  23]
 [ 36 130  44]
 [160  56 143]]
